# MONAI Auto3DSeg "Hello World" Example

In this notebook, we will provide a simple demonstration of how to use Auto3DSeg AutoRunner to process a simulated dataset and generate results within minutes.

<img src="https://raw.githubusercontent.com/Project-MONAI/tutorials/main/auto3dseg/figures/workflow_v1.png" width="60%" />

## Overview

3D medical image segmentation is an important task with great potential for clinical understanding, disease diagnosis, and surgical planning. Auto3DSeg is a comprehensive solution for large-scale 3D medical image segmentation. It leverages the latest advances in MONAI and GPUs to efficiently develop and deploy algorithms with state-of-the-art performance for beginners or advanced researchers in the field. According to the statistics of the recent MICCAI conferences, more than 60% of the papers are applications of segmentation algorithms, and more than half of them use 3D datasets. After working in this field for many years, we have released the state-of-the-art segmentation solution Auto3DSeg, which requires minimal user input (e.g., data root and list).

Auto3DSeg first analyzes the global information, such as the dataset's intensity, data size, and data spacing. Then it generates algorithm folders in MONAI bundle format based on data statistics and algorithm templates. Next, all algorithms initiate model training to obtain checkpoints with the best validation accuracy. Finally, the ensemble module selects the algorithms via ranking trained checkpoints and creates ensemble predictions. Meanwhile, the solution offers different levels of user experience for beginners and advanced researchers. It has been tested on large-scale 3D medical imaging datasets in several different modalities.

- Unified Framework: Auto3DSeg is a self-contained 3D medical image segmentation solution with minimal user input.
- Flexible Modular Design: Auto3DSeg components can be used independently to meet different needs of users.
- Support of Bring-Your-Own-Algorithm (BYOA): We have introduced an efficient way to introduce users' own algorithms into the Auto3DSeg framework.
- High Accuracy and Efficiency: Auto3DSeg achieves state-of-the-art performance in most applications of 3D medical image segmentation.


### Table of Contents
* [Imports](#0.-Imports)
* [Dataset Setup](#1.-Dataset-Setup)
* [Auto3DSeg](#2.-Auto3DSeg)
* [Inspect Predictions](#3.-Inspect-Predictions)
* [Optional Exercises](#4.-Optional-Exercises)
* [Conclusion](#5.-Conclusion)

### System Setup

Before proceeding with the example, it is important to ensure that the NVIDIA driver has been installed correctly. You can verify this by running the nvidia-smi command, which should provide information about the GPUs on your system.

In [ ]:
!nvidia-smi

### 0. Imports

In this notebook, we will only require minimal imports from MONAI and utilities, as Auto3DSeg will handle the remaining imports during its various phases.

#### 0.1 Installation
Before running the notebook, we need to ensure that the required packages are installed in the environment. You can install the necessary packages by running the following commands:

In [ ]:
!python -c "import monai" || pip install -U "monai[fire, nibabel, yaml, tqdm, einops]==1.2"
!python -c "import yaml" || pip install -U "pyyaml"
!python -c "import psutil" || pip install -U "psutil"
!python -c "import tensorboard" || pip install -U "tensorboard"

#### 0.2 Imports

Let's import the required libraries and use the `print_config()` function to ensure that everything is installed correctly.

In [ ]:
import os
import json
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt

from monai.apps.auto3dseg import AutoRunner
from monai.config import print_config
from monai.data import create_test_image_3d

print_config()

### 1. Dataset Setup

#### 1.1 Simulate a Small Dataset

Training AI models can be time-consuming, so to provide a quick demonstration of Auto3DSeg in this notebook, we will simulate a small dataset and train the model for only a few epochs. Keep in mind that the performance of the trained model may not be optimal due to the limited training time, but the entire pipeline will be completed within minutes.

The `sim_datalist` variable contains information about the simulated datasets. It includes a list of 24 training images and corresponding labels, as well as 8 testing images and labels. The training data is divided into 2 folds, where each fold uses 12 images for training and 4 images for validation. The size of the dimensions can be defined by the `sim_dim` variable.

To proceed with the dataset setup, we will follow the steps below:

In [ ]:
sim_datalist = {
    "testing": [
        {"image": "test_image_001.nii.gz", "label": "test_label_001.nii.gz"},
        {"image": "test_image_002.nii.gz", "label": "test_label_002.nii.gz"},
        {"image": "test_image_003.nii.gz", "label": "test_label_003.nii.gz"},
        {"image": "test_image_004.nii.gz", "label": "test_label_004.nii.gz"},
        {"image": "test_image_005.nii.gz", "label": "test_label_005.nii.gz"},
        {"image": "test_image_006.nii.gz", "label": "test_label_006.nii.gz"},
        {"image": "test_image_007.nii.gz", "label": "test_label_007.nii.gz"},
        {"image": "test_image_008.nii.gz", "label": "test_label_008.nii.gz"},

    ],
    "training": [
        {"fold": 0, "image": "tr_image_001.nii.gz", "label": "tr_label_001.nii.gz"},
        {"fold": 0, "image": "tr_image_002.nii.gz", "label": "tr_label_002.nii.gz"},
        {"fold": 0, "image": "tr_image_003.nii.gz", "label": "tr_label_003.nii.gz"},
        {"fold": 0, "image": "tr_image_004.nii.gz", "label": "tr_label_004.nii.gz"},
        {"fold": 0, "image": "tr_image_005.nii.gz", "label": "tr_label_005.nii.gz"},
        {"fold": 0, "image": "tr_image_006.nii.gz", "label": "tr_label_006.nii.gz"},
        {"fold": 0, "image": "tr_image_007.nii.gz", "label": "tr_label_007.nii.gz"},
        {"fold": 0, "image": "tr_image_008.nii.gz", "label": "tr_label_008.nii.gz"},
        {"fold": 0, "image": "tr_image_009.nii.gz", "label": "tr_label_009.nii.gz"},
        {"fold": 0, "image": "tr_image_010.nii.gz", "label": "tr_label_010.nii.gz"},
        {"fold": 0, "image": "tr_image_011.nii.gz", "label": "tr_label_011.nii.gz"},
        {"fold": 0, "image": "tr_image_012.nii.gz", "label": "tr_label_012.nii.gz"},
        {"fold": 1, "image": "tr_image_013.nii.gz", "label": "tr_label_013.nii.gz"},
        {"fold": 1, "image": "tr_image_014.nii.gz", "label": "tr_label_014.nii.gz"},
        {"fold": 1, "image": "tr_image_015.nii.gz", "label": "tr_label_015.nii.gz"},
        {"fold": 1, "image": "tr_image_016.nii.gz", "label": "tr_label_016.nii.gz"},
        {"fold": 1, "image": "tr_image_017.nii.gz", "label": "tr_label_017.nii.gz"},
        {"fold": 1, "image": "tr_image_018.nii.gz", "label": "tr_label_018.nii.gz"},
        {"fold": 1, "image": "tr_image_019.nii.gz", "label": "tr_label_019.nii.gz"},
        {"fold": 1, "image": "tr_image_020.nii.gz", "label": "tr_label_020.nii.gz"},
        {"fold": 1, "image": "tr_image_021.nii.gz", "label": "tr_label_021.nii.gz"},
        {"fold": 1, "image": "tr_image_022.nii.gz", "label": "tr_label_022.nii.gz"},
        {"fold": 1, "image": "tr_image_023.nii.gz", "label": "tr_label_023.nii.gz"},
        {"fold": 1, "image": "tr_image_024.nii.gz", "label": "tr_label_024.nii.gz"},
    ],
}

sim_dim = (64, 64, 64)

#### 1.2 Generate Images and Labels

We can utilize the `create_test_image_3d` function from MONAI and `nib.Nifti1Image` from the nibabel library to generate the 3D simulated images. These images will be saved under the specified `work_dir`.

Since we have already defined our image set, we can iterate over those images and label names to create matching files. In this case, our simulated data will consist of random circles generated in a 3D space, which we will visualize below.

In [ ]:
work_dir = "./helloworld_work_dir"
if not os.path.isdir(work_dir):
    os.makedirs(work_dir)

dataroot_dir = os.path.join(work_dir, "sim_dataroot")
if not os.path.isdir(dataroot_dir):
    os.makedirs(dataroot_dir)

datalist_file = os.path.join(work_dir, "sim_datalist.json")
with open(datalist_file, "w") as f:
    json.dump(sim_datalist, f)

for d in sim_datalist["testing"] + sim_datalist["training"]:
    im, seg = create_test_image_3d(
        sim_dim[0], sim_dim[1], sim_dim[2], rad_max=10, num_seg_classes=1, random_state=np.random.RandomState(42)
    )
    image_fpath = os.path.join(dataroot_dir, d["image"])
    label_fpath = os.path.join(dataroot_dir, d["label"])
    nib.save(nib.Nifti1Image(im, affine=np.eye(4)), image_fpath)
    nib.save(nib.Nifti1Image(seg, affine=np.eye(4)), label_fpath)

#### 1.3 Inspect the Simulated Data

To get an idea of the simulated images in the test dataset, we can select a slice from the 3D image and visualize it. This will give us a glimpse of the simulated data.

In [ ]:
img_nib = nib.load(os.path.join(dataroot_dir, sim_datalist["testing"][0]["image"]))
lbl_nib = nib.load(os.path.join(dataroot_dir, sim_datalist["testing"][0]["label"]))
img = np.array(img_nib.dataobj)
lbl = np.array(lbl_nib.dataobj)
plt.subplot(1, 2, 1)
plt.imshow(img[32])
plt.title("image")
cbar = plt.colorbar(shrink=0.55)
plt.subplot(1, 2, 2)
plt.imshow(lbl[32])
plt.title("label")
cbar = plt.colorbar(shrink=0.55)

## 2. Auto3DSeg
#### 2.1 Using the Auto3DSeg AutoRunner Pipeline

The `AutoRunner` class provides an interface to run the Auto3DSeg pipeline. It offers high configurability and requires minimal user input to get started. The `AutoRunner` automatically configures multiple neural networks and performs multi-fold training to achieve state-of-the-art performance. In this example, we aim to complete the pipeline within two to four minutes, so we need to make some customizations.

Let's set up the `AutoRunner` with the following settings:

In [ ]:
runner = AutoRunner(
    work_dir=work_dir,
    input={
        "modality": "MRI",
        "datalist": datalist_file,
        "dataroot": dataroot_dir,
    }
)

#### 2.2 Override the Training Parameters

If you wish to use more than one GPU, you can modify the `CUDA_VISIBLE_DEVICES` environment variable or remove the key to utilize all available devices. However, please ensure that the number of GPUs is less than the number of partitions in the dataset to avoid any conflicts.

In this section, you can customize the settings for the `AutoRunner` to align with your specific use case. For this example, we will use smaller numbers to explore the functionality and utilization of Auto3DSeg. However, in real-world scenarios, you should adjust these numbers based on the dataset size, task difficulty, and the number of available GPUs.

In [ ]:
max_epochs = 2

train_param = {
    "num_epochs_per_validation": 1,
    "num_images_per_batch": 2,
    "num_epochs": max_epochs,
    "num_warmup_epochs": 1,
}
runner.set_training_params(train_param)
runner.set_num_fold(num_fold=1)

#### 2.3 Starting the Auto3DSeg Pipeline

After setting the desired parameters, we can start the Auto3DSeg pipeline by using the `runner.run()` method. The `AutoRunner` will execute all the necessary steps to train the model.

Each module of Auto3DSeg can be used individually for different purposes, and the functions/methods within the components can be customized by users.

The Auto3DSeg pipeline consists of the following steps:

- Step 1: [Data Analyzer](https://github.com/Project-MONAI/tutorials/blob/main/auto3dseg/docs/data_analyzer.md)
- Step 2: [Algorithm Generation](https://github.com/Project-MONAI/tutorials/blob/main/auto3dseg/docs/algorithm_generation.md)
- Step 3: [Model Training, Validation, and Inference](https://github.com/Project-MONAI/tutorials/blob/main/auto3dseg/docs/bundle.md)
- Step 4 (Optional): [Hyper-parameter Optimization](https://github.com/Project-MONAI/tutorials/blob/main/auto3dseg/docs/hpo.md)
- Step 5: [Model Ensembling](https://github.com/Project-MONAI/tutorials/blob/main/auto3dseg/docs/ensemble.md)

In [ ]:
runner.run()

## 3. Inspecting Predictions

Now that we have completed the Auto3DSeg pipeline and obtained the ensembled output, we can inspect the predictions and compare the results.

In [ ]:
image_name = sim_datalist["testing"][0]["image"].split(".")[0]
prediction_nib = nib.load(os.path.join(work_dir, "ensemble_output", image_name + "_ensemble" + ".nii.gz"))
pred = np.array(prediction_nib.dataobj)

img_slice32 = lbl[32] == 0
label_slice32 = lbl[32] == 1
background_slice32 = pred[32, :, :, 0] if pred.ndim == 4 else pred[32] == 0
foreground_slice32 = pred[32, :, :, 1] if pred.ndim == 4 else pred[32] == 1

plt.subplot(2, 2, 1)
plt.imshow(img_slice32)
plt.title("background groundtruth")
cbar = plt.colorbar(shrink=0.8)
plt.subplot(2, 2, 2)
plt.imshow(label_slice32)
plt.title("foreground_groundtruth")
cbar = plt.colorbar(shrink=0.8)
plt.subplot(2, 2, 3)
plt.imshow(background_slice32)
plt.title("background prediction")
cbar = plt.colorbar(shrink=0.8)
plt.subplot(2, 2, 4)
plt.imshow(foreground_slice32)
plt.title("foreground prediction")
cbar = plt.colorbar(shrink=0.8)
# set the spacing between subplots
plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4, hspace=0.4)
plt.show()

## 4. Optional Exercises
The optional exercises provide additional tasks for those who want to explore and go beyond the scope of the tutorial.

#### 4.1 Additional Algorithms and Longer Training
In our example above, we used training parameters to make sure that we could run the example in roughly two minutes.  However, you can experiment with different algorithms and longer training durations using the existing dataset.

As you explore more options, you'll notice the ability to customize various parameters depending on the algorithms. For example, the DiNTS network has two phases: a Search phase and a Train phase. You can independently modify the parameters for each of these sub-sections within DiNTS.

#### 4.2 Use a Real Dataset
In our "Hello World" example, we utilized a synthetic dataset to explore the functionalities of Auto3DSeg. However, the true benefits of Auto3DSeg can be seen when using it with real-world datasets.

We provide an additional notebook called `msd_datalist_generator.ipynb` in the sidebar, which can help generate the `datalist.json` for any of the Medical Segmentation Decathlon datasets. You can leverage MONAI's wrapper classes to retrieve any of the 10 different segmentation tasks from the Medical Segmentation Decathlon dataset and apply the power of Auto3DSeg to the task.


#### 4.3 Hyper-Parameter Optimization
We introduced some basic options for parameter selection in Auto3DSeg. However, you can further explore a more robust hyper-parameter optimization (HPO) experience by utilizing one of the supported third-party HPO packages.

We provide a base HPO generator class called `HPOGen`, which supports interactions between our algorithms and third-party HPO packages such as Microsoft Neural Network Intelligence (NNI) and Optuna.

You can refer to the following tutorials to explore HPO with Auto3DSeg:
- [NNI Tutorial](https://github.com/Project-MONAI/tutorials/blob/main/auto3dseg/notebooks/hpo_nni.ipynb)
- [Optuna Tutorial](https://github.com/Project-MONAI/tutorials/blob/main/auto3dseg/notebooks/hpo_optuna.ipynb)

#### 4.4 Implement Each Step of the Pipeline

In this notebook, we utilized the `AutoRunner` class to launch the Auto3DSeg pipeline. However, this class is just a wrapper for multiple internal API calls. You can explore calling each of these APIs and modifying additional parameters during each phase.

We provide a notebook called `auto3dseg_autorunner_ref_api.ipynb` that walks you through how to call each of the internal API functions within AutoRunner. This allows you to have more fine-grained control over the pipeline and customize parameters during each step.

Feel free to explore these optional exercises to further enhance your understanding and utilization of Auto3DSeg.

## 5. Conclusion
Congratulations! You have completed the entire Auto3DSeg pipeline. Throughout the process, you observed the algorithms learning from the dataset and generating predictions for both background and foreground voxels. 

Auto3DSeg and AutoRunner offer high configurability, allowing you to customize various aspects of the pipeline. To improve your results, you can consider increasing the training time, exploring different ensemble methods, or utilizing hyper-parameter optimization through the AutoRunner or Auto3DSeg module APIs.

If you're interested in real-world use cases of Auto3DSeg, you can find more information and tutorials in the [Auto3DSeg Tutorials Folder](https://github.com/Project-MONAI/tutorials/tree/main/auto3dseg/).

Feel free to explore and experiment with Auto3DSeg to tackle your own 3D medical image segmentation tasks. Happy exploring!